### ДЗ 2.1

In [1]:
%load_ext pycodestyle_magic
%flake8_on

#### 1. Выбрать и сохранить книгу в .txt
Выбрана книга "Белая гвардия", сохранена в txt в кодировке utf-8.

#### 2. Обработать книгу с помощью mystem:
##### 2.1. Распарсить с помощью mystem
##### 2.2. Замерить время работы

In [12]:
from pymystem3 import Mystem

m = Mystem()
file_name = "white_guardian.txt"
with open(file_name, encoding='utf-8') as f:
    text = f.read()

Удалим знаки пунктуации с помощью функции translate, транслируя каждый символ из пунктуации в ничто. Символы конца строки превратим в пробелы.

In [13]:
import string

punct_chars = string.punctuation
text = text.translate({ord(ch): None for ch in punct_chars})
text = text.replace('\n', ' ')

Проанализируем и замерим время:

In [14]:
%%time
analysis = m.analyze(text)

Wall time: 6.18 s


##### 2.3. Сохранить результат в json
В прошлый раз использовали load() для импорта, теперь используем dump() для экспорта:

In [15]:
import json

with open('mystem_analysis.json', 'w', encoding='utf-8') as f:
    json.dump(analysis, f, ensure_ascii=False)

#### 3. Обработать книгу через pymorphy
##### 3.1. Токенизировать текст с помощью nltk
##### 3.2. Замерить время работы
Импортируем word_tokenize. Здесь может появиться ошибка типа [этой](https://github.com/joosthub/PyTorchNLPBook/issues/14), поступаем, как сказано в ответах к ишью.

In [22]:
from nltk.tokenize import word_tokenize

tokenized_text = word_tokenize(text)

Создадим экземпляр класса MorphAnalyzer:

In [16]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Отпарсим все слова:

In [23]:
%%time
parsed_words = []

for word in tokenized_text:
    parsed_word = morph.parse(word)
    parsed_words.append(parsed_word)

Wall time: 22.8 s


##### 3.3, 3.4. Сохранить результат (хотя бы лемма + часть речи) в json
Создадим список, куда свалим все разборы. А сами разборы преобразуем в словари по тэгам "в лоб".

In [52]:
list_of_parsing = []

for word in parsed_words:
    w = word[0]
    entry = {
        'word': w.normal_form,
        'pos': w.tag.POS,  # Part of Speech, часть речи
        'animacy': w.tag.animacy,  # одушевленность
        'aspect': w.tag.aspect,  # вид: совершенный или несовершенный
        'case': w.tag.case,  # падеж
        'gender': w.tag.gender,  # род (мужской, женский, средний)
        'involvement': w.tag.involvement,  # включенность говорящего в действие
        'mood': w.tag.mood,  # наклонение (повелительное, изъявительное)
        'number': w.tag.number,  # число (единственное, множественное)
        'person': w.tag.person,  # лицо (1, 2, 3)
        'tense': w.tag.tense,  # время (настоящее, прошедшее, будущее)
        'transitivity': w.tag.transitivity,  # переходность
        'voice': w.tag.voice,  # залог (действительный, страдательный)
    }
    list_of_parsing.append(entry)

Сложим в json:

In [53]:
file_name = "list_of_parsing.json"
with open(file_name, 'w', encoding='utf-8') as f:
    json.dump(list_of_parsing, f, ensure_ascii=False)

#### 4. Ответить на вопросы:
##### 4.1. Какую долю слов составляет каждая часть речи? (Например, для глагола - это количество глаголов, деленное на общее число слов в тексте)
Сложим части речи в отдельный список и посчитаем все вхождения с помощью Counter, заодно отсортировав их для пущей наглядности с помощью most_common().

In [90]:
from collections import Counter

parts_of_speech = []

for parsing in list_of_parsing:
    parts_of_speech.append(parsing['pos'])

pos_count = Counter(parts_of_speech).most_common()
total_count = len(parts_of_speech)

for k, v in pos_count:
    print(k, v/total_count*100)

NOUN 29.973306215838313
VERB 14.013530500120053
ADJF 11.390760278519272
PREP 10.882307246868072
CONJ 9.954380464104629
ADVB 6.025168425066735
NPRO 5.276612572913577
PRCL 4.320438399502846
INFN 1.6468228747369462
PRTF 1.2513594056749007
GRND 1.1976893634450516
ADJS 1.0734008445969805
NUMR 0.7965764162535485
None 0.5508241176221346
PRED 0.48726748866573455
PRTS 0.47314379334209
INTJ 0.4011129471915032
COMP 0.28529864553761847


##### 4.2. Найдите топ-20 (по частотности) глаголов и наречий
Снова используем тот же подход к аггрегации и сортировке, а затем выведем топ 20:

In [97]:
list_of_verbs = []
list_of_adverbs = []

for parsing in list_of_parsing:
    if 'VERB' == parsing['pos']:
        list_of_verbs.append(parsing['word'])
    if 'ADVB' == parsing['pos']:
        list_of_adverbs.append(parsing['word'])

verbs_count = Counter(list_of_verbs).most_common()
adverbs_count = Counter(list_of_adverbs).most_common()

print("Top 20 verbs:\n")
for k, v in verbs_count[:20]:
    print(k, v)

print("\nTop 20 adverbs:\n")
for k, v in adverbs_count[:20]:
    print(k, v)

Top 20 verbs:

быть 669
сказать 204
стать 165
мочь 152
говорить 126
идти 99
ответить 94
знать 91
спросить 63
видеть 56
выйти 55
думать 54
дать 51
бежать 45
пройти 45
исчезнуть 42
сидеть 42
начать 41
ходить 41
подумать 41

Top 20 adverbs:

уже 140
очень 102
тут 99
где 97
там 95
потом 91
сейчас 87
совершенно 81
потому 71
вдруг 68
теперь 58
здесь 50
затем 50
опять 46
уж 42
тогда 40
совсем 38
сразу 38
сегодня 32
несколько 32


#### 5. Посмотрите документацию для nltk н-грамм (nltk.bigrams, например), попробуйте составить топ-25 биграмм и триграмм для вашего текста в лемматизированном виде (только леммы, без знаков препинания). Почему получаются именно такие?

Сложим все нормальные формы в список:

In [111]:
lemma_list = []
for parsing in list_of_parsing:
    lemma_list.append(parsing['word'])

Импортируем модули и считаем:

In [115]:
from nltk.util import bigrams
bigram_list = Counter(list(bigrams(lemma_list))).most_common()

for bigram in bigram_list[:25]:
    print(bigram[0], bigram[1])

('и', 'в') 102
('не', 'быть') 82
('в', 'город') 76
('и', 'не') 68
('что', 'он') 66
('потому', 'что') 65
('не', 'мочь') 56
('у', 'он') 51
('мочь', 'быть') 50
('он', 'и') 49
('господин', 'полковник') 48
('он', 'не') 45
('не', 'знать') 43
('он', 'в') 43
('он', 'быть') 41
('что', 'вы') 40
('ничто', 'не') 39
('николка', 'и') 38
('и', 'на') 36
('и', 'весь') 36
('в', 'глаз') 34
('и', 'он') 33
('тот', 'что') 33
('в', 'чёрный') 32
('в', 'столовый') 32


In [117]:
from nltk.util import trigrams
trigram_list = Counter(list(trigrams(lemma_list))).most_common()

for trigram in trigram_list[:25]:
    print(trigram[0], trigram[1])

('что', 'же', 'это') 14
('у', 'он', 'быть') 12
('ни', 'в', 'кой') 11
('в', 'кой', 'случай') 11
('в', 'город', 'и') 11
('не', 'мочь', 'быть') 10
('не', 'знать', 'что') 9
('в', 'это', 'время') 9
('в', 'тот', 'же') 9
('черта', 'он', 'знать') 8
('о', 'тот', 'что') 8
('до', 'сей', 'пора') 7
('в', 'три', 'час') 7
('я', 'думать', 'что') 7
('до', 'тот', 'пора') 7
('и', 'не', 'быть') 7
('в', 'конец', 'конец') 7
('один', 'из', 'они') 7
('слушать', 'господин', 'полковник') 7
('у', 'он', 'на') 7
('и', 'сейчас', 'же') 7
('в', 'самый', 'дело') 6
('в', 'всякий', 'случай') 6
('мочь', 'быть', 'и') 6
('кой', 'случай', 'не') 6


Здесь любопытны:
* триграмма "что же это", указывающая на невероятность и трагичность описываемых в книге событий;
* биграммы и триграммы со словами "город", косвенно намекающие, что город - квази-персонаж романа;
* "господин полковник" и "слушать господин полковник" - результат, по-видимому, намеренного употребления этого выражения по отношению к герою романа полковнику Малышеву (это, возможно, оттого, что это реальное лицо под своей фамилией)